In [10]:
import tensorflow as tf
import pandas as pd

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.orders.criteria import StopLoss, StopDirection
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import LogDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 1
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = LogDifference(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

action_scheme = ManagedRiskOrders(pairs=[USD/BTC])
reward_scheme = RiskAdjustedReturns(return_algorithm="sortino")

# csv_file = tf.keras.utils.get_file(
#     'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
ohlcv_data = pd.read_csv('./data/Coinbase_BTCUSD_1h.csv', skiprows=1, index_col="date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})

exchange = SimulatedExchange(data_frame=ohlcv_data, price_column=PRICE_COLUMN, randomize_time_slices=True)

wallets = [(exchange, USD, 10000), (exchange, BTC, 0)]

# portfolio = Portfolio(base_instrument=USD, wallet_tuples=wallets)
portfolio = Portfolio(base_instrument=USD, wallets=wallets)


environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE,
                                 observe_wallets=[USD, BTC])

print('Observation Data:')
print(environment.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

strategy.run(steps=128)

Observation Data:
['open' 'high' 'low' 'close' 'volume_btc' 'volume' 'USD' 'BTC'
 'USD_pending' 'BTC_pending']
Execute:  <Order: id=9f3590cc-b3d9-42d7-ad32-fd41ab801abe, status=0, type=market, side=buy, pair=USD/BTC, quantity=9000.00 USD, size=9000.0, price=8051.0, criteria=None, path_id=9f3590cc-b3d9-42d7-ad32-fd41ab801abe | Recipes: <Recipe: id=b2bf8c3c-8153-419a-b5b5-db8541bb71a4, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.03 (T/P) | 0.06 (S/L)>>
Fill:  <Trade: id=4f99d67b-1238-4627-8dc7-44da1edd4fcc, order_id=9f3590cc-b3d9-42d7-ad32-fd41ab801abe, step=1, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=8973.00 USD, price=8051.0, commission=27.00 USD>
Total traded:  9000.0
Completed:  9f3590cc-b3d9-42d7-ad32-fd41ab801abe <Order: id=d2f509f9-3a53-4998-b842-ac795c73bcd1, status=0, type=market, side=sell, pair=USD/BTC, quantity=1.11451994 BTC, size=8973.0, price=8051.0, criteria=StopLoss: either | 0.03 (T/P) | 0.06 (S/L), path_id=9f3590cc-b3d9-42d7-ad32-

Execute:  <Order: id=d2f509f9-3a53-4998-b842-ac795c73bcd1, status=0, type=market, side=sell, pair=USD/BTC, quantity=1.11451994 BTC, size=8973.0, price=8051.0, criteria=StopLoss: either | 0.03 (T/P) | 0.06 (S/L), path_id=9f3590cc-b3d9-42d7-ad32-fd41ab801abe | Recipes: >
Fill:  <Trade: id=812eeff2-285a-4090-9556-f10677d7fed5, order_id=d2f509f9-3a53-4998-b842-ac795c73bcd1, step=42, base_symbol=USD, quote_symbol=BTC, side=sell, type=market, quantity=8946.08 USD, price=8320.0, commission=26.92 USD>
Total traded:  8973.0
Completed:  d2f509f9-3a53-4998-b842-ac795c73bcd1 None
Execute:  <Order: id=1ea1e6d8-decc-428b-afa5-6070b235d546, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.07500103 BTC, size=598.2, price=7975.89, criteria=StopLoss: either | 0.03 (T/P) | 0.06 (S/L), path_id=1859c0be-eb2e-4377-b8ae-8df5ca40d995 | Recipes: >
Fill:  <Trade: id=5c08f441-ba9b-45de-852b-f3495db4241c, order_id=1ea1e6d8-decc-428b-afa5-6070b235d546, step=42, base_symbol=USD, quote_symbol=BTC, side=sel

Execute:  <Order: id=f5a013f1-4c1e-4127-ac19-67e2e7ea59e2, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.00582788 BTC, size=47.43, price=8138.47, criteria=StopLoss: either | 0.03 (T/P) | 0.02 (S/L), path_id=d87ef7a7-a4f2-4737-8e0f-1348672e533d | Recipes: >
Fill:  <Trade: id=94027466-b004-4561-bb0c-3a3b607e9d17, order_id=f5a013f1-4c1e-4127-ac19-67e2e7ea59e2, step=86, base_symbol=USD, quote_symbol=BTC, side=sell, type=market, quantity=47.29 USD, price=8446.01, commission=0.14 USD>
Total traded:  47.43
Completed:  f5a013f1-4c1e-4127-ac19-67e2e7ea59e2 None
Execute:  <Order: id=03cce361-f044-4b96-afa3-db5c927145de, status=0, type=market, side=sell, pair=USD/BTC, quantity=1.07131701 BTC, size=8848.5, price=8259.46, criteria=StopLoss: either | 0.02 (T/P) | 0.04 (S/L), path_id=1f11dd25-cc32-44db-aa77-0603ee53bc39 | Recipes: >
Fill:  <Trade: id=a1e82df4-6a92-4cb0-80af-8f0ee40489ba, order_id=03cce361-f044-4b96-afa3-db5c927145de, step=86, base_symbol=USD, quote_symbol=BTC, side=sell,

Execute:  <Order: id=6665898d-e79a-4b56-a29d-b95be52a82dc, status=0, type=market, side=buy, pair=USD/BTC, quantity=3.10 USD, size=3.1, price=8348.16, criteria=None, path_id=6665898d-e79a-4b56-a29d-b95be52a82dc | Recipes: <Recipe: id=0469f1e4-ad08-4b30-92f8-ee53d5cd143e, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.02 (T/P) | 0.04 (S/L)>>
Fill:  <Trade: id=76b33597-1e43-483d-8335-af8a0da66b59, order_id=6665898d-e79a-4b56-a29d-b95be52a82dc, step=96, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=3.09 USD, price=8348.16, commission=0.01 USD>
Total traded:  3.1
Completed:  6665898d-e79a-4b56-a29d-b95be52a82dc <Order: id=68e6a74c-bb96-4f5b-b975-864e621c517b, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.00037014 BTC, size=3.09, price=8348.16, criteria=StopLoss: either | 0.02 (T/P) | 0.04 (S/L), path_id=6665898d-e79a-4b56-a29d-b95be52a82dc | Recipes: >
Execute:  <Order: id=ab7ccdeb-c367-4db5-9ef1-edac8a6cce5c, status=0, type=market, side=buy, pair

,BTC,BTC_pending,USD,USD_pending,net_worth,step
0,0.000000,1.114520,1000.00,0.0,9973.000037,1
0,0.000000,1.189521,400.00,0.0,9887.488409,2
0,0.000000,1.214557,200.00,0.0,9873.482504,3
0,0.000000,1.237034,20.00,0.0,9896.504915,4
0,0.000000,1.239545,0.00,0.0,9844.105969,5
0,0.000000,1.239545,0.00,0.0,9879.408207,6
0,0.000000,1.239545,0.00,0.0,9899.773930,7
0,0.000000,1.239545,0.00,0.0,9866.777245,8
0,0.000000,1.239545,0.00,0.0,9874.747518,9
0,0.000000,1.239545,0.00,0.0,9908.351580,10
